In [3]:
import json
import pandas as pd

In [5]:
df = pd.read_csv("data/2022/input/skf/predictions_stratified/darknet_features_20221021_fold0.csv")

In [6]:
df.head()

,src_ip,censys,driftnet,internetcensus,intrinsec,ipip,mirai,onyphe,rapid7,securitytrails,shadowserver,shodan,u_mich,unk_bruteforcer,unk_exploiter,unk_spammer,unknown,y_true
0,97.74.81.123,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,1.000000,unknown
1,212.70.149.14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,1.000000,unknown
2,171.22.30.185,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.666667,0.0,0.333333,0.000000,unknown
3,195.3.147.57,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.000000,0.666667,unk_bruteforcer
4,111.7.96.132,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,1.000000,unknown


In [7]:
df.y_true.value_counts()

y_true
mirai              8041
unknown            4096
unk_spammer         492
shadowserver        289
unk_bruteforcer     278
driftnet            252
internetcensus      226
censys              110
onyphe               97
shodan               26
securitytrails       18
ipip                 12
intrinsec            12
unk_exploiter        10
u_mich               10
Name: count, dtype: int64

In [8]:
df = pd.read_csv("data/2022/input/stacking_predictions/out/k3/train/idarkvec_20221021_fold00.csv")

In [9]:
df.y_true.value_counts()

y_true
mirai              7237
unknown            4096
unk_spammer         443
shadowserver        260
unk_bruteforcer     250
driftnet            227
internetcensus      203
censys               99
onyphe               88
shodan               23
securitytrails       16
ipip                 11
intrinsec            11
unk_exploiter         9
u_mich                9
Name: count, dtype: int64

In [10]:
df = pd.read_csv(
    "data/2022/input/stacking_predictions/out/k3/test/idarkvec_20221021_fold00.csv"
)

In [11]:
df.y_true.value_counts()

y_true
mirai              804
unk_spammer         49
shadowserver        29
unk_bruteforcer     28
driftnet            25
internetcensus      23
censys              11
onyphe               9
shodan               3
securitytrails       2
ipip                 1
intrinsec            1
u_mich               1
unk_exploiter        1
Name: count, dtype: int64

In [4]:
with open("new_data/skf/stratification/stratification.json", "r") as fd:
    splits = json.load(fd)

In [5]:
splits.keys()

dict_keys(['20221021', '20221022', '20221023', '20221024', '20221025', '20221026', '20221027', '20221028', '20221029', '20221030', '20221031'])

In [6]:
splits["20221021"]["darknet"][0][0][:3]

['97.74.81.123', '212.70.149.14', '171.22.30.185']

In [7]:
splits["20221021"].keys()

dict_keys(['darknet', 'honeypot'])

In [8]:
splits["20221021"]["honeypot"][0][1][:3]

['117.187.173.104', '222.185.146.149', '117.196.109.167']

In [9]:
probs_cols = [
    "censys",
    "driftnet",
    "internetcensus",
    "intrinsec",
    "ipip",
    "mirai",
    "onyphe",
    "rapid7",
    "securitytrails",
    "shadowserver",
    "shodan",
    "u_mich",
    "unk_bruteforcer",
    "unk_exploiter",
    "unk_spammer",
    "unknown",
]

probs_cols.sort()
probs_cols

['censys',
 'driftnet',
 'internetcensus',
 'intrinsec',
 'ipip',
 'mirai',
 'onyphe',
 'rapid7',
 'securitytrails',
 'shadowserver',
 'shodan',
 'u_mich',
 'unk_bruteforcer',
 'unk_exploiter',
 'unk_spammer',
 'unknown']

In [10]:
df_train = df[df.src_ip.isin(splits["20221021"]["darknet"][0][0])]
df_test = df[df.src_ip.isin(splits["20221021"]["darknet"][0][1])]

In [11]:
def get_X_y(df, probs_col):
    X = df[probs_col].copy(deep=True)
    y = df.y_true
    return X, y

In [12]:
X_train, y_train = get_X_y(df_train, probs_cols)
X_test, y_test = get_X_y(df_test, probs_cols)

In [13]:
X_train.shape, X_test.shape

((12982, 16), (987, 16))